# Imports


In [54]:
# For Data
import numpy as np
import pandas as pd
import re
from datetime import datetime
from tqdm.notebook import tqdm

#  For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as pyo 
import plotly.graph_objects as go
import plotly.figure_factory as ff
import missingno as msno
from wordcloud import WordCloud
import random

# For models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier


# For NLP
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from spellchecker import SpellChecker

import pickle
from imblearn.over_sampling import SMOTE
from collections import Counter

# For Styling
plt.style.use('fivethirtyeight')

# Downloading periphrals
nltk.download('vader_lexicon')
nltk.download('stopwords')

from preprocessing import *
from plot import *
from feature_extractor import *
from data_balance import *
from model import *

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/mostafawael/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mostafawael/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Import the dataset

In [55]:
train_file = 'Dataset/train.csv'
devFile = 'Dataset/dev.csv'
train_df = pd.read_csv(train_file)
dev_df = pd.read_csv(devFile)
print(f"Training dataset size = {train_df.shape}")
print(f"Dev dataset size = {dev_df.shape}")

Training dataset size = (6988, 3)
Dev dataset size = (1000, 3)


# Data Preprocessing

In [56]:
# Data cleaning
training_data = cleanData(train_df, 'training', clean = False, clearData = True)
print(f"Cleaned Training dataset size = {training_data.shape}")
# Data cleaning
dev_data = cleanData(dev_df, 'dev', clean = False, clearData = True)
print(f"Cleaned Dev dataset size = {dev_data.shape}")

Cleaned Training dataset size = (6988, 3)
Cleaned Dev dataset size = (1000, 3)


In [57]:
# Some preprocessing steps, like extracting limmitization
training_data = training_data.pipe(processing)    
print(f"Processed Training dataset size = {training_data.shape}")
print(training_data.columns)
# Some preprocessing steps, like extracting limmitization
dev_data = processing(dev_data)   
print(f"Processed dev dataset size = {dev_data.shape}")
print(dev_data.columns)

Processed Training dataset size = (6988, 5)
Index(['text', 'category', 'stance', 'Lemmatization', 'sentiment'], dtype='object')
Processed dev dataset size = (1000, 5)
Index(['text', 'category', 'stance', 'Lemmatization', 'sentiment'], dtype='object')


In [58]:
# Save the out
training_data.to_csv('out/training_data_processed.csv', index=False) # print the df in a csv file
# Save the out
dev_data.to_csv('out/dev_data_processed.csv', index=False) # print the df in a csv file

# Feature Engineering

In [59]:
## Word2Vec embeddings
model = extractWordEmbeddings(training_data['Lemmatization']) # use word2vec to extract the word embeddings
training_data['features'] = getTweetsEmbeddings(model, training_data['Lemmatization']) # get the word embeddings for each tweet
trainingFeatures = training_data['features'].to_numpy()

## Word2Vec embeddings
model = extractWordEmbeddings(dev_data['Lemmatization']) # use word2vec to extract the word embeddings
dev_data['features'] = getTweetsEmbeddings(model, dev_data['Lemmatization']) # get the word embeddings for each tweet
devFeatures = dev_data['features'].to_numpy()


[WARNING] [2022-12-25 22:01:37,440:gensim.models.word2vec] Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
[WARNING] [2022-12-25 22:01:38,611:gensim.models.word2vec] Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.


In [60]:
# XLM embeddings
# features = np.load('out/train_embeddings.npy')
# XLM embeddings
# dev_data['features'] = np.load('out/test_embeddings.npy').tolist()

# Balancing the training dataset

In [61]:
# convert features into 2d array
trainingFeatures = np.array([np.array(xi) for xi in trainingFeatures])
stances = training_data['stance'].to_numpy()
columns = ["f" + str(i + 1) for i in range(len(trainingFeatures[0]))]

df = pd.DataFrame(trainingFeatures, columns=columns)
df = pd.DataFrame(trainingFeatures)
df['stance'] = stances

X_train, y_train = balance_data(df)
print("Some notes about dimensions of the data")
print(f"X_train size before cleaning = {trainingFeatures.shape}")
print(f"X_train size = {X_train.shape}")
print(f"y_train size = {y_train.shape}")

Before balancing:
Class=2, n=5538 (79.250%)
Class=1, n=1012 (14.482%)
Class=0, n=438 (6.268%)
After balancing:
Class=2, n=5538 (33.333%)
Class=1, n=5538 (33.333%)
Class=0, n=5538 (33.333%)
Some notes about dimensions of the data
X_train size before cleaning = (6988, 100)
X_train size = (16614, 100)
y_train size = (16614,)


In [62]:
split = 'y'
if split == 'y':
    # Split the data into train and test
    X_train, X_test, y_train, y_test = train_test_split(trainingFeatures, stances, test_size=0.2, random_state=42)
else:
    X_test = np.array([np.array(xi) for xi in dev_data['features'].to_numpy()])
    # X_test = np.load('out/test_embeddings.npy')
    y_test = dev_data['stance'].to_numpy()

print(f"X_test size = {X_test.shape}")
print(f"y_test size = {y_test.shape}")
# print unique values in the dataset
print(f"Unique values in the dataset = {np.unique(y_test)}")
print(f"Unique values in the dataset = {np.unique(y_train)}")

X_test size = (1398, 100)
y_test size = (1398,)
Unique values in the dataset = [-1  0  1]
Unique values in the dataset = [-1  0  1]


# Model Building
**Build a multi-class classifier to predict the category of the tweet**

In [63]:
# RandomForestClassifier
clf = RandomForestClassifier(n_estimators=500, max_depth=15, random_state=0)
_y_train = y_train + 1
_y_test = y_test + 1
model, report = modelPipeline(X_train, y_train, X_test, y_test, clf, 'out/models/clf.model')
report

              precision    recall  f1-score   support

          -1       0.50      0.04      0.08        93
           0       0.42      0.08      0.13       208
           1       0.80      0.98      0.88      1097

    accuracy                           0.78      1398
   macro avg       0.57      0.37      0.36      1398
weighted avg       0.72      0.78      0.71      1398



0.7839771101573677

In [64]:
# Xgboost
xgb = XGBClassifier()
_y_train = y_train + 1
_y_test = y_test + 1
model, report = modelPipeline(X_train, _y_train, X_test, _y_test, xgb, 'out/models/xgb.model')
report

              precision    recall  f1-score   support

           0       0.33      0.08      0.12        93
           1       0.33      0.08      0.12       208
           2       0.80      0.97      0.87      1097

    accuracy                           0.78      1398
   macro avg       0.49      0.37      0.37      1398
weighted avg       0.70      0.78      0.71      1398



0.7753934191702432

In [65]:
# Naive Bayes
gnb = GaussianNB(var_smoothing=10)
model, report = modelPipeline(X_train, y_train, X_test, y_test, gnb, 'out/models/gnb.model')
report

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        93
           0       0.00      0.00      0.00       208
           1       0.78      1.00      0.88      1097

    accuracy                           0.78      1398
   macro avg       0.26      0.33      0.29      1398
weighted avg       0.62      0.78      0.69      1398



0.7846924177396281

In [66]:
# SVM
svm = svm.SVC()
model, report = modelPipeline(X_train, y_train, X_test, y_test, svm, 'out/models/svm.model')
report

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        93
           0       0.00      0.00      0.00       208
           1       0.78      1.00      0.88      1097

    accuracy                           0.78      1398
   macro avg       0.26      0.33      0.29      1398
weighted avg       0.62      0.78      0.69      1398



0.7846924177396281

# Auto ML
Check out the [Auto SKlearn](https://automl.github.io/auto-sklearn/master/index.html)

In [67]:
import autosklearn.classification
import sklearn.datasets
import sklearn.metrics
from sklearn.utils.multiclass import type_of_target

In [68]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=1900, # overall time in seconds
    per_run_time_limit=1300, # time per model in seconds
    initial_configurations_via_metalearning=0,
    ensemble_size=10,
    n_jobs=8,
    smac_scenario_args={"runcount_limit": 1},
)
automl.fit(X_train, y_train)
y_pred = automl.predict(X_test)

[WARNING] [2022-12-25 22:02:19,413:Client-AutoML(1):093be1d2-848f-11ed-983d-1da6501c908a] Capping the per_run_time_limit to 949.0 to have time for a least 2 models in each process.


In [69]:
print(automl.sprint_statistics())
print(automl.leaderboard())

auto-sklearn results:
  Dataset name: 093be1d2-848f-11ed-983d-1da6501c908a
  Metric: accuracy
  Best validation score: 0.795664
  Number of target algorithm runs: 1
  Number of successful target algorithm runs: 1
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0

          rank  ensemble_weight           type      cost   duration
model_id                                                           
2            1              1.0  random_forest  0.204336  20.671741


In [70]:
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_pred)) #  get the Score of the final ensemble

Accuracy score 0.7761087267525035
